HATE DETECTION : H2H

In [1]:
import pandas as pd
df = pd.read_csv("train.csv")

In [2]:
df

id                                       comment_text  \
0       0000997932d777bf  Explanation\nWhy the edits made under my usern...   
1       000103f0d9cfb60f  D'aww! He matches this background colour I'm s...   
2       000113f07ec002fd  Hey man, I'm really not trying to edit war. It...   
3       0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...   
4       0001d958c54c6e35  You, sir, are my hero. Any chance you remember...   
...                  ...                                                ...   
159566  ffe987279560d7ff  ":::::And for the second time of asking, when ...   
159567  ffea4adeee384e90  You should be ashamed of yourself \n\nThat is ...   
159568  ffee36eab5c267c9  Spitzer \n\nUmm, theres no actual article for ...   
159569  fff125370e4aaaf3  And it looks like it was actually you who put ...   
159570  fff46fc426af1f9a  "\nAnd ... I really don't think you understand...   

        toxic  severe_toxic  obscene  threat  insult  identity_hate  
0           0             0        0       0       0              0  
1           0             0        0       0       0              0  
2           0             0        0       0       0              0  
3           0             0        0       0       0              0  
4           0             0        0       0       0              0  
...       ...           ...      ...     ...     ...            ...  
159566      0             0        0       0       0              0  
159567      0             0        0       0       0              0  
159568      0             0        0       0       0              0  
159569      0             0        0       0       0              0  
159570      0             0        0       0       0              0  

[159571 rows x 8 columns]

In [3]:
import torch
import numpy as np
Y = []
for i in df.values:
    Y.append(torch.tensor(np.array(i[2:],dtype = np.float32)))
Y = torch.stack(Y)
print(Y)
Y.shape

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])


torch.Size([159571, 6])

EMBEDDINGS OF TWEETS

In [4]:
import pickle
import pandas as pd
import json
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
model_sen = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
X = []
for i in tqdm(df["comment_text"]):
    j = torch.tensor(model_sen.encode(i))
    X.append(j)
X = torch.stack(X)

100%|██████████| 159571/159571 [8:54:03<00:00,  4.98it/s]   


In [ ]:
#saving the embeddings for future use
with open('X.pkl', 'wb') as f:
    pickle.dump(X, f)

print("Stacked tensor saved to X.pkl")

Stacked tensor saved to stacked_tensor.pkl


In [9]:
import torch
import pickle

# Load the embeddings from the pickle file
with open('X.pkl', 'rb') as f:
    X = pickle.load(f)
X = X.unsqueeze(1)
# Print the loaded tensor
print(X)


tensor([[[ 0.0610, -0.0387, -0.0026,  ..., -0.0268, -0.0665, -0.0274]],

        [[-0.0121, -0.0163, -0.0072,  ...,  0.0131, -0.0092, -0.0323]],

        [[ 0.0751,  0.0372, -0.0001,  ...,  0.0099, -0.0469, -0.0610]],

        ...,

        [[ 0.0199,  0.1087,  0.0155,  ..., -0.0486, -0.0388,  0.0154]],

        [[ 0.0585, -0.0155, -0.0265,  ...,  0.0055, -0.0276,  0.0078]],

        [[ 0.0355,  0.0207, -0.0079,  ...,  0.0050,  0.0335, -0.0203]]])


In [6]:
X.shape

torch.Size([159571, 1, 768])

In [10]:
# combing X and Y to form a dataset
import tensorflow as tf
dataset = tf.data.Dataset.from_tensor_slices((X,Y))
dataset = dataset.cache()
dataset = dataset.shuffle(1000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(5)

In [11]:
train = dataset.take(int(len(dataset)*.6))
val = dataset.skip(int(len(dataset)*.6)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.8)).take(int(len(dataset)*.2))

In [36]:
train_generator = train.as_numpy_iterator()
train_generator.next()

(array([[[ 0.07982665,  0.02901868,  0.02951389, ..., -0.02089134,
          -0.0537919 , -0.00722066]],
 
        [[ 0.06077495,  0.08171018, -0.00139317, ..., -0.00273512,
          -0.01272774, -0.01402025]],
 
        [[-0.05156567,  0.08263606, -0.00082603, ..., -0.00329034,
          -0.02862681, -0.00517731]],
 
        ...,
 
        [[-0.01306925, -0.0190337 , -0.01882688, ...,  0.04338536,
           0.0144974 ,  0.01738532]],
 
        [[ 0.07737225, -0.04597089, -0.01692977, ...,  0.01313443,
          -0.05814308, -0.02379304]],
 
        [[ 0.04133406,  0.03415545, -0.00478682, ...,  0.01761078,
          -0.00388963, -0.09994441]]], dtype=float32),
 array([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0

MODEL

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Activation
import torch
# Define the Keras model
model = Sequential([
    Bidirectional(LSTM(768, activation = 'tanh')),
    Dense(512),
    Activation('tanh'),
    Dense(256),
    Activation('tanh'),
    Dense(128),
    Activation('tanh'),
    Dense(16),
    Activation('tanh'),
    Dense(6),
    Activation('sigmoid')
])
# model1(X_text_train.unsqueeze(1))

# Compile the model
# optimizer = tf.keras.optimizers.Adam(lr=0.01)
model.compile(loss='binary_crossentropy', optimizer= 'Adam')

# Print the model summary
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [12]:
history = model.fit(train, epochs=1, validation_data= val)

5984/5984 ━━━━━━━━━━━━━━━━━━━━ 987s 163ms/step - loss: 0.1514 - val_loss: 0.0806


In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 1536)           │     9,443,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       786,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │         2,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 6)              │           102 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 6)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,189,988 (118.98 MB)

 Trainable params: 10,396,662 (39.66 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 20,793,326 (79.32 MB)

PREDICTIONS

In [17]:
X_test = torch.tensor(model_sen.encode("I'll kill you")).unsqueeze(0).unsqueeze(0)
X_test

tensor([[[ 7.1044e-02,  4.8869e-02, -5.5874e-03,  4.4661e-03, -6.5701e-03,
           3.8937e-02, -3.0228e-02,  4.2765e-02, -1.2309e-02,  1.3392e-02,
           2.7514e-02,  2.2837e-02,  7.3720e-02,  5.7936e-02, -6.5131e-03,
           6.2170e-03,  1.0437e-02, -5.4102e-03,  5.1908e-02, -2.3087e-02,
          -5.3402e-03,  2.7886e-02, -6.2808e-03,  1.0456e-02,  1.0376e-02,
           1.3865e-02,  5.3587e-02,  2.5933e-02, -3.4686e-03, -3.3729e-02,
          -9.2243e-03, -4.4367e-02, -4.2160e-02,  2.2287e-02,  1.7971e-06,
          -4.2737e-02,  6.8327e-02, -1.1885e-03, -5.4989e-02,  1.0131e-01,
           4.8786e-03, -7.5937e-02, -2.4148e-02, -3.0298e-02,  1.2081e-02,
          -2.1161e-02,  9.9955e-03,  2.6894e-02,  2.1409e-02, -5.1982e-03,
          -8.0993e-03, -5.3718e-02,  2.8362e-02, -2.2426e-02,  7.7232e-03,
           1.9633e-02,  1.5691e-02,  3.1930e-02, -1.3382e-02, -3.5782e-03,
          -2.1913e-02, -7.3918e-03, -1.0127e-02,  4.6985e-02,  8.1481e-03,
           1.1517e-03,  5

In [18]:
Y_test = model.predict(X_test)
Y_test

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


array([[0.82261217, 0.06877942, 0.55911183, 0.0204431 , 0.5009505 ,
        0.08734142]], dtype=float32)

In [20]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [14]:
test_batch_X, test_batch_Y = test.as_numpy_iterator().next()

In [15]:
predict_Y = (model.predict(test_batch_X) > 0.5).astype(int)
predict_Y

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [16]:
test_batch_Y

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]], dtype=float32)

EVALUATING THE MODEL

In [17]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(test_batch_Y, predict_Y, average='micro')
recall = recall_score(test_batch_Y, predict_Y, average= 'micro')
accuracy = (predict_Y == test_batch_Y).astype(float).mean().item()
print("accuracy: ",accuracy)
print("recall: ",recall)
print("precision: ",precision)

accuracy:  0.9895833333333334
recall:  1.0
precision:  0.75


SAVE MODEL

In [42]:
from tensorflow.keras.models import save_model

save_model(model, 'hate_detection.keras')


In [ ]:
from tensorflow.keras.models import load_model

# Load the model
loaded_model = load_model('hate_detection.keras')


In [19]:
def evaluate_hate(tweet):
    eval_X = torch.tensor(model_sen.encode(tweet)).unsqueeze(0).unsqueeze(0)
    res = model.predict(eval_X)
    result = f'tweet: "{tweet}"\n'
    for i, col in enumerate(df.columns[2:]):
        result += f'{col}: {res[0][i]>0.5}\n'
    return result

TEST HERE

In [21]:
print("WELCOME TO HATE DETECTOR: H2H")
tweet = input("Enter the tweet to check")
print(evaluate_hate(tweet))

WELCOME TO HATE DETECTOR: H2H
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
tweet: "I hate you, you piece of shit"
toxic: True
severe_toxic: False
obscene: True
threat: False
insult: True
identity_hate: False

